In [1]:
# https://github.com/sudarshan-koirala/youtube-stuffs/blob/main/langchain/langchain_Semi_Structured_RAG.ipynb

In [37]:
import os
import re

from unstructured.partition.pdf import partition_pdf

from langchain.chat_models import AzureChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains.question_answering import load_qa_chain
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore, create_kv_docstore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.schema.document import Document
# Set logging for the queries
import logging
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.memory import VectorStoreRetrieverMemory
from langchain.globals import set_debug
set_debug(False)

from operator import itemgetter

from collections import defaultdict

from typing import Any, List
from pydantic import BaseModel, Field

# 前置參數

- 公司proxy設定

In [38]:
# proxy相關
proxy = "auhqproxy.cdn.corpnet.auo.com:8080"
auo_account = "benbllee"
auo_password = "Aa0937454850"

# proxy加入環境變數
http_proxy_url = f"http://{auo_account}:{auo_password}@{proxy}"
https_proxy_url = f"https://{auo_account}:{auo_password}@{proxy}"
os.environ["HTTP_PROXY"] = http_proxy_url 
os.environ["HTTPS_PROXY"] = https_proxy_url

- 欲解析的pdf名稱

In [39]:
pdf_name = "202302_2409_AIA_20231105_140505.pdf"
# pdf_name = "react.pdf"
assert os.path.exists(pdf_name)

- unstructured要做partition，需要先在本機安裝poppler、Tesseract-OCR，並設置環境變數

參考網址: https://github.com/insightbuilder/python_de_learners_data/blob/main/code_script_notebooks/projects/langChain_exploration/Unstructured_Quick_Tour_withPyPdf.ipynb

In [40]:
# tesseract安裝路徑
tesseract_install_dir = "C:\\Users\\BenBLLee\\AppData\\Local\\Programs\\Tesseract-OCR"
assert os.path.exists(tesseract_install_dir)

# poppler的 work dir路徑
poppler_bin_dir = "poppler-23.10.0\\Library\\bin"
assert os.path.exists(poppler_bin_dir)

# 將poppler、tesseract位置加入環境變數
os.environ["PATH"] += os.pathsep + os.path.join(os.getcwd(), poppler_bin_dir) + os.pathsep + tesseract_install_dir

- 使用本機的tiktoken模型

參考網址: https://stackoverflow.com/questions/76106366/how-to-use-tiktoken-in-offline-mode-computer

In [41]:
# tiktoken本地模型的位置
tiktoken_cache_dir = os.path.join(os.getcwd(),"9b5ad71b2ce5302211f9c61530b329a4922fc6a4")
assert os.path.exists(tiktoken_cache_dir)

# tiktoken本地模型的位置加入環境變數
os.environ["TIKTOKEN_CACHE_DIR"] = tiktoken_cache_dir

- openai相關設定

openai若設置正確，但仍無法取得api回應，需先取得crt認證。參考網址: https://community.openai.com/t/ssl-certificate-verify-failed/32442/39?page=2

In [42]:
# openai認證
openai_crt_dir = "openaiCert/TMGCert.crt"
os.environ['REQUESTS_CA_BUNDLE'] = os.path.join(os.getcwd(), openai_crt_dir)

# openai type
openai_type = "azure"

# Azure GPT url, api-key
chatgpt_url = "https://auoazchatgpt08.openai.azure.com/openai/deployments/GPT4/chat/completions?api-version=2023-07-01-preview"
chatgpt_key = "e498c3852b1145ddb27ff0b9cbc4f4ca"

# Azre GPT url中，包含base_url、deployment_name的資料
pattern = r"(https://.+\.openai\.azure\.com/)openai/deployments/(.+)/chat/.+api-version=(.+)"
gpt_base_url = re.search(pattern, chatgpt_url).group(1)            # ex. "https://auoazchatgpt03.openai.azure.com/"
gpt_deployment = re.search(pattern, chatgpt_url).group(2)     # ex. "GPT4"
gpt_version = re.search(pattern, chatgpt_url).group(3)             # ex. "2023-07-01-preview"

- 建立langchain的chatopenai model

In [43]:
# chat model
model = AzureChatOpenAI(
    openai_api_base=gpt_base_url,
    openai_api_version=gpt_version,
    deployment_name=gpt_deployment,
    openai_api_key=chatgpt_key,
    openai_api_type=openai_type,
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler()],
    temperature=0,
)

# Retriever

https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector#summary

Use Multi Vector Retriever with summaries:

- InMemoryStore stores the raw text, tables
- vectorstore stores the embedded summaries

- 用hugging face上的 embedding model建立chroma vector db

In [44]:
# embedding model
embedd_model = "BAAI/bge-large-en-v1.5" # 英文
# embedd_model = "BAAI/bge-small-zh" # 中文

embeddings = HuggingFaceEmbeddings(model_name=embedd_model)

In [45]:
vector_store_dir = "./vector store"
collection_name=f'{pdf_name.replace(".pdf","")}.vectorstore'

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings,
    persist_directory=vector_store_dir,
    collection_metadata={"hnsw:space": "cosine"}
)


- 將text的檔案建在本地資料夾中
MultiVectorRetriever要吃BaseStore類

In [46]:
# The storage layer for the parent documents
file_store_dir = f"./text store/{pdf_name.replace('.pdf','')}"
store = LocalFileStore(root_path=file_store_dir) # 讓mset的資料可存本地BaseStore interface that works on the local file system.
doc_store = create_kv_docstore(store=store) # mset的key-value的value變成可放入Document的資料

- 建立MultiVectorRetriever
向量搜尋相似vector，依vector的id_key返回該id_key的doc_store文檔

In [47]:
id_key = "page"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=doc_store,
    id_key=id_key,
    search_kwargs={'k':3, "score_threshold": 1}
)




- 建立Multi Query Retriever，把user輸入的question,映射出其它類似的questions

In [48]:
# logging.basicConfig()
# logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)
m_q_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=model
)

- 建立ContextualCompressionRetriever，取得文檔時，先用user question對文檔壓縮，僅保留與question相關的資訊

In [49]:
compressor = LLMChainExtractor.from_llm(llm=model)
comp_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=m_q_retriever)

# Unstructured PDF

用預訓練模型對pdf頁面的元素切塊，解析出text、table

參考資料:
1. https://www.youtube.com/watch?v=AYBMbIMG19M&t=34s
2. https://github.com/sudarshan-koirala/youtube-stuffs/blob/main/langchain/langchain_Semi_Structured_RAG.ipynb

- 解析pdf

In [50]:
raw_pdf_elements = partition_pdf(filename=pdf_name,
                                 # Unstructured first finds embedded image blocks
                                 extract_images_in_pdf=False,
                                 # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
                                 # Titles are any sub-section of the document
                                 infer_table_structure=True,
                                 # Post processing to aggregate text once we have the title
                                 chunking_strategy="by_title",
                                 # Chunking params to aggregate text blocks
                                 # Attempt to create a new chunk 3800 chars
                                 # Attempt to keep chunks > 2000 chars
                                 max_characters=4000,
                                 new_after_n_chars=3800,
                                 combine_text_under_n_chars=2000,
                                 #image_output_dir_path=path
                                 )

- 用pydantic整理Unstructure解析的pdf元素，先分開text、table元素

In [51]:
# 定義Element class
class Element(BaseModel):
    type: str = Field(..., description="元素的類型。表格型元素為'table'，文字型元素為'text'")
    text: str = Field(..., description="元素的內文。文字型元素為單純的text，表格元素則為html_text")
    page: int = Field(..., description="元素的所在頁數。")
    summary: str = Field("", description="元素內文的摘要。")
    hypo_questions: List[str] = Field([], description="3個假想可用元素的內文來回答的假設性問題。")

# Categorize element's type, page. Put text content into Element
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=element.metadata.text_as_html, page=element.metadata.page_number))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=element.text, page=element.metadata.page_number))

# Table elements
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text elements
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

126
136


# embedding
Smaller chunks: 目前pdf已經被unstructure拆成更小的元素

可以以下兩取向則一執行:

1. 取向一: 用small chunk的摘要做後續embedding
2. 取向二: 用small chunk的假設性問題做後續embedding

參考資料:

https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector#summary

https://python.langchain.com/docs/modules/chains/how_to/openai_functions


- 取向一: 用small chunk的摘要做後續embedding

In [52]:
# 將table、text的文字取出為串列
tables = [e.text for e in table_elements]
texts = [e.text for e in text_elements]

In [53]:
summary_prompt_text= \
"""You are an assistant tasked with summarizing tables and text. \
Give a concise summary of the table or text. Table or text chunk: {content} """

summary_prompt = ChatPromptTemplate.from_template(summary_prompt_text)

summarize_chain = (
    {"content":RunnablePassthrough()}    
    | summary_prompt 
    | model 
    | StrOutputParser()
    )

# tables元素做摘要
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

# texts元素做摘要
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

The table provides a detailed breakdown of current and noncurrent assets. Current assets include cash and cash equivalents, financial assets at fairThe table provides a detailed breakdown of the company's cash flows from operating activities. It shows a loss before income tax of $15,557,410The table provides aThe table presents a detailed financial summary for the years 2022 and 2023. It includes various financial metrics such as revenue, sales return detailed breakdown of liabilities and equity for a company as of June 30, 2023, and December 31, 2022. The largestThe table provides a detailed breakdown of the financial status of AUO Corporation from January 1, 2022, to June 30, 2023. It includes and discount, net revenue, cost of sales, gross profit, operating expenses, selling and distribution expenses, general and administrative expenses, research and in the first period and a profit of $2,371,190 in the second period. The table also lists various expenses and gains such as depreciat

- 取向二: 用small chunk的假設性問題做後續embedding

In [54]:
# 用pydantic取得假設性問題的摘要
functions = [
    {
        "name": "hypothetical_questions",
        "description": "Generate hypothetical questions",
        "parameters": {
            "type": "object",
            "properties": {
                "questions": {
                    "type": "array",
                    "items": {"type": "string"},
                },
            },
            "required": ["questions"],
        },
    }
]

# 獲取假設性問題的prompt
hpo_q_format_prompt =ChatPromptTemplate.from_template(
    "Generate a list of 3 hypothetical questions that the below document could be used to answer:\n\n{doc}"
)
    

get_hypo_q_chain = (
    RunnablePassthrough() | hpo_q_format_prompt
    | model.bind(functions=functions, function_call={"name": "hypothetical_questions"})
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

# tables元素做摘要
table_questions = get_hypo_q_chain.batch([ {"doc":e.text} for e in table_elements], {"max_concurrency": 5})
# texts元素做摘要
text_questions = get_hypo_q_chain.batch([ {"doc":e.text} for e in text_elements], {"max_concurrency": 5})

- summary、hypo_questions合併回Elements

In [55]:
for e, s, q in list(zip(table_elements, table_summaries, table_questions)):
    e.summary = s
    e.hypo_questions = q

for e, s, q in list(zip(text_elements, text_summaries, text_questions)):
    e.summary = s
    e.hypo_questions = q

- 新增summary和hypoquestions的Document到vector store

In [56]:
def save_elements_doc_to_vectorstore(elements):    
    
    for e in elements:

        page = str( e.page)

        # 待新增的Document list
        docs = [] 

        # summary的Document
        e_summary_doc = Document(page_content=e.text, metadata={id_key: page})
        docs.append(e_summary_doc)

        for q in e.hypo_questions:

            # 假設性問題的Document
            e_question_doc = Document(page_content=q, metadata={id_key: page})
            docs.append(e_question_doc)
        
        # 存入vectorstore
        retriever.vectorstore.add_documents(docs)

    # vectorstore save to local disk
    retriever.vectorstore.persist()

# 將tables、texts的元素存入vectorstore
save_elements_doc_to_vectorstore(text_elements+table_elements)

- 新增每頁的原文(text+table)到filesotre

In [57]:
# Add texts and tables to docstore
def save_page_content_to_filestore(elements):
    page_doc_dic = defaultdict(lambda: Document(page_content="", metadata={id_key: ""}))
    for e in elements:
        page = str(e.page)
        page_doc_dic[page].page_content += "\n\n" + e.text
        page_doc_dic[page].metadata[id_key] = page

    retriever.docstore.mset( list(page_doc_dic.items()))

# 將tables、texts的元素存入filesotre
save_page_content_to_filestore(text_elements+table_elements)

# Memory
- VectorStoreRetrieverMemory

In [85]:
memory_key="history"
memory_input_key="question"

# 建立vector store
memory_vectorstore = Chroma(
    collection_name="conversation_memory",
    embedding_function=embeddings,
    collection_metadata={"hnsw:space": "cosine"}
)

# 實體化VectorStoreRetrieverMemory
memory = VectorStoreRetrieverMemory(retriever=memory_vectorstore.as_retriever(), memory_key=memory_key, input_key=memory_input_key)

# 提取相關對話歷史
def get_relavant_memory_history(question):
    return memory.load_memory_variables({memory_input_key:question})[memory_key]

# 儲存本次對話歷史
def save_to_memory_history(kwargs:{"question":str, "model_result":dict}):
    question = kwargs.get("question")
    model_result = kwargs.get("model_result")
    memory.save_context({"input":question}, {"output":model_result["output_text"]})
    return model_result


# RAG from LangChain Expression Language.


定義鏈: 將使用者問題延伸做相似搜尋，依回傳頁數的Document、使用者問題做refine摘要

- prompt

In [86]:
# 使用者提問結合對話歷史脈絡的prompt
condense_question_prompt = ChatPromptTemplate.from_messages([
        ("system", "以下提供與客戶對話歷史中，可能與客戶需求相關的訊息。請留下對話歷史中與客戶需求有關的訊息，並彙整成一個單獨的問題描述(翻譯成英文)，讓後續處理同仁可以單獨看到此問題描述後就明確得知客戶需求及過去對話脈絡，並有效找到回應此客戶需求的文檔。"),
        ("human", "對話歷史:'{history}'\n\n客戶需求:'{question}'"),
])

# refine最開始前，第一份文檔要先做此prompt
initial_abstract_prompt = ChatPromptTemplate.from_messages([
            ("system", "你是一個撰寫報告或部落格的寫手，請你基於客戶需求，從客戶提供的文本脈絡用繁體中文來回答可回應需求的文章。文章需寫成類似分析報告的形式，要依客戶需求展開列點、補充內容，尤其要多用文章脈絡所提之事實、數據佐證。但若文本脈絡無法呼應問題，請不用回答任何文字。"),
            ("human", "文本脈絡:\n\n{context_str}\n\n需求:{condensed_question}"),
])

# refine的第二份文檔以後，用此prompt
refine_abstract_prompt = ChatPromptTemplate.from_messages([
            ("system", "請完成給你的已摘要的文章，並基於提供給你的文本脈絡，用繁體中文來修改或接續已摘要的文章，以回應客戶需求。文章需寫成類似分析報告的形式，要依客戶需求展開列點、補充內容，尤其要多用文章脈絡所提之事實、數據佐證。但若文本脈絡無法呼應問題，請保留或微調原摘要即可。"),
            ("human", "已摘要的文章:{existing_answer}\n\n你這次拿到的文本脈絡:\n\n{context_str}\n\n客戶需求:{condensed_question}"),
])

- 定義摘要對話歷史及問題的chain

In [87]:
condense_question_chain = (
    {"question":RunnablePassthrough(),
    "condensed_question":{"question":RunnablePassthrough(), "history":RunnablePassthrough()| get_relavant_memory_history } 
                        | condense_question_prompt 
                        | model 
                        | StrOutputParser()
    }
)

- 定義獲取document的chain

In [88]:
# multiple query retriever -> docs
mq_retrieve_doc_chain = (
    RunnablePassthrough()
    |{"question":itemgetter("question"),
     "condensed_question":itemgetter("condensed_question"),
    "input_documents": itemgetter("question")| m_q_retriever 
    }
) 

# multiple query retriever -> docs -> compressed docs
cp_retrieve_doc_chain = (
    RunnablePassthrough()
    |{"question":itemgetter("question"),
     "condensed_question":itemgetter("condensed_question"),
    "input_documents": itemgetter("condensed_question")| m_q_retriever 
    }
) 

- 定義qa_chain

In [89]:
# langchain已有封裝好的load_qa_chain
qa_model = load_qa_chain(model, chain_type="refine", return_refine_steps=True, question_prompt=initial_abstract_prompt, refine_prompt=refine_abstract_prompt)
qa_chain = (
    RunnablePassthrough()
    |{"question":itemgetter("question"),
      "result":{"condensed_question":itemgetter("condensed_question"),"input_documents":itemgetter("input_documents")}|qa_model
    }
)

- 儲存對話歷史的chain

In [90]:
memory_save_chain = (
    RunnablePassthrough()
    |{"question":itemgetter("question"),"model_result":itemgetter("result")} 
    | save_to_memory_history
)

In [91]:

mq_chain = (
    condense_question_chain # 將對話歷史結合使用者提問
    | mq_retrieve_doc_chain # 使用者提問提取相關文件
    | qa_chain # 依據提問及提供的文件做摘要回答
    | memory_save_chain # 將回答存入對話歷史
)

cp_chain = (
    condense_question_chain # 將對話歷史結合使用者提問
    | cp_retrieve_doc_chain # 使用者提問提取相關文件
    | qa_chain # 依據提問及提供的文件做摘要回答
    | memory_save_chain # 將回答存入對話歷史
)



In [92]:
# mq_result = mq_chain.invoke("幫我分析AUO的基本面", config={"callbacks": [ConsoleCallbackHandler()]})

In [93]:
cp_result = cp_chain.invoke("幫我分析AUO的基本面", config={"callbacks": [ConsoleCallbackHandler()]})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "幫我分析AUO的基本面"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "幫我分析AUO的基本面"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "幫我分析AUO的基本面"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "幫我分析AUO的基本面"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "幫我分析AUO的基本面"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnableSequence > 5:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "幫我分析AUO的基本面"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnableSequence > 5:chain:RunnablePara

In [94]:
print(cp_result['output_text'])

根據提供的文本脈絡，我們可以看到友達光電(AU Optronics Corp., AUO)的一些子公司和其主要業務，以及其財務狀況。以下是對AUO基本面的分析：

1. 子公司與業務範疇：AUO擁有多個子公司，分布在中國、美國、日本、加拿大和英國等地。這些子公司的業務範疇廣泛，包括銷售和銷售支援活動、系統設計、電子組件製造、租賃活動、智能醫療服務、建設項目和相關項目管理、軟硬體銷售和諮詢服務等。這種多元化的業務結構有助於AUO在不同市場和產業中獲得穩定的收入。

2. 持股比例：根據表格，AUO在大部分子公司中都持有100%的股份，顯示出其在這些子公司的業務中具有決定性的影響力。此外，有兩個子公司，即Heilongjiang Talenda Smart Display Technology Co., Ltd.和ProfetAI (Suzhou) Co., Ltd.，AUO的持股比例為51%，這意味著AUO在這兩家公司中也具有控股地位。

3. 業務變化：從表格中可以看出，有一些子公司在2022年6月30日和2022年12月31日的持股比例為100%，但在2023年6月30日變為0%，這可能意味著AUO在這段期間出售了這些子公司的股份。這種變化可能是AUO根據其業務策略和市場環境的變化進行的調整。

4. 財務狀況：根據財務報表，AUO在2023年和2022年的前六個月都虧損，且虧損額在2023年比2022年更大。這可能反映出AUO在這段期間的業務環境有所變化，或者其業務策略需要進行調整。此外，AUO在2023年上半年的長期借款增加，這可能是為了應對業務變化或者進行投資。

5. 公允價值衡量：AUO的管理層每年會至少審查一次公允價值衡量的政策和程序，或者在需要的時候更頻繁地審查。當公允價值衡量涉及到一個或多個重要的不可觀察輸入時，公司會謹慎地監控評價過程，並檢查是否使用了最相關的市場數據。

6. 資產和負債的變動：根據財務報表，AUO在2023年6月30日的公允價值透過其他綜合收益衡量的權益工具（FVTOCI - equity instruments）餘額為647,120，比2022年的1,160,309有所減少。這可能反映出AUO在這段期間的投資策略有所變化。

總的來說，AUO具有多元化的業務結構和在多數子公司中的控股地位，這可能有助於其在不同市場和產業中獲得

In [97]:
cp_result = cp_chain.invoke("你前面說AUO在2023年6月30日的公允價值透過其他綜合收益衡量的權益工具（FVTOCI - equity instruments）餘額為647,120。請問647,120的單位是?", config={"callbacks": [ConsoleCallbackHandler()]})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "你前面說AUO在2023年6月30日的公允價值透過其他綜合收益衡量的權益工具（FVTOCI - equity instruments）餘額為647,120。請問647,120的單位是?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "你前面說AUO在2023年6月30日的公允價值透過其他綜合收益衡量的權益工具（FVTOCI - equity instruments）餘額為647,120。請問647,120的單位是?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "你前面說AUO在2023年6月30日的公允價值透過其他綜合收益衡量的權益工具（FVTOCI - equity instruments）餘額為647,120。請問647,120的單位是?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "你前面說AUO在2023年6月30日的公允價值透過其他綜合收益衡量的權益工具（FVTOCI - equity instruments）餘額為647,120。請問647,120的單位是?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnableSequence] Entering Chain run with input:
{
 

In [98]:
print(cp_result['output_text'])

根據您提供的文本脈絡，我們可以看到AUO公司的財務報表中，有許多數據都以新台幣（NTD）的千元為單位。然而，對於您詢問的2023年6月30日的其他綜合收益公允價值（FVTOCI）-權益工具餘額，該文本並未明確提及其數值為647,120的單位。

然而，考慮到財務報表中的其他數據大多以新台幣的千元為單位，我們可以推測，這個647,120的數值可能也是以新台幣的千元為單位。換句話說，這個數值可能代表的是647,120,000新台幣。

然而，這只是一個推測，並不能確定。為了獲得確切的答案，我們建議您可以直接向AUO公司或相關財務專業人士查詢。

此外，根據文本脈絡，我們可以看到AUO公司在2023年6月30日的應收帳款餘額為19,878,140千元，並且該公司在該日期的損失準備金為2,389千元。這些數據可能對您理解AUO公司的財務狀況有所幫助。

總結來說，我們可以從以下幾點來理解AUO公司的財務狀況：

1. 2023年6月30日的其他綜合收益公允價值（FVTOCI）-權益工具餘額可能為647,120,000新台幣，但需要進一步確認。

2. 2023年6月30日的應收帳款餘額為19,878,140千元，顯示公司有一定的資金流入。

3. 2023年6月30日的損失準備金為2,389千元，顯示公司對於可能的風險有所預防。

以上分析僅供參考，如需更詳細的資訊，建議直接向AUO公司或相關財務專業人士查詢。


In [95]:
cp_result = cp_chain.invoke("是否能幫我依照AUO的持股比例(由多至少)列出所有子公司的清單", config={"callbacks": [ConsoleCallbackHandler()]})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "是否能幫我依照AUO的持股比例(由多至少)列出所有子公司的清單"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "是否能幫我依照AUO的持股比例(由多至少)列出所有子公司的清單"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "是否能幫我依照AUO的持股比例(由多至少)列出所有子公司的清單"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "是否能幫我依照AUO的持股比例(由多至少)列出所有子公司的清單"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "是否能幫我依照AUO的持股比例(由多至少)列出所有子公司的清單"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnableSequence > 5:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "是否能幫我依照AUO的持股比例(由多至少)列出所有子公司的清單

In [96]:
print(cp_result['output_text'])

根據客戶的需求，以下是AUO的子公司列表，並按照AUO的持股比例從高到低排序：

1. AUO Display Plus Netherlands B.V. (ADPNL) - 持股比例：100.00%
   主要業務：持股，銷售和銷售支援活動 (荷蘭)

2. AUO Green Energy America Corp. (AEUS) - 持股比例：100.00%
   主要業務：銷售支援活動 (美國)

3. Evergen Power Corporation (EGPC) - 持股比例：100.00%
   主要業務：太陽能發電 (台灣)

4. Sungen Power Corporation (SGPC) - 持股比例：100.00%
   主要業務：太陽能發電 (馬來西亞)

5. BriView (L) Corp. - 持股比例：100.00%
   主要業務：持股公司 (美國)

6. AUO (Kunshan) Co., Ltd. - 持股比例：100.00%
   主要業務：製造和銷售公司 (新加坡)

7. AUO (Slovakia) s.r.0 (AUOSK, formerly AU Optronics (Slovakia) s.r.o.) - 持股比例：100.00%
   主要業務：修理活動 (斯洛伐克共和國)

8. AUO Manufacturing (Shanghai) Co., Ltd. (AUOSJ) - 持股比例：100.00%
   主要業務：租賃活動 (中國)

9. AUO (Suzhou) Co., Ltd. | (AUOSZ) - 持股比例：100.00%
    主要業務：製造和銷售公司 (中國)

10. AUO (Xiamen) Co., Ltd. © (AUOXM) - 持股比例：100.00%
    主要業務：製造和銷售公司 (中國)

11. AUO (Shanghai) Co., Ltd. (AUOSH) - 持股比例：100.00%
    主要業務：銷售支援活動 (中國)

12. AUO Singapore Pte. Ltd. | (AUOSG) - 持股比例：100.00%
    主要業務：持股公司和銷售支援活動 (新加坡)

13. AUO Korea Ltd. (AUOKR, forme